In [ ]:
import os
from os import path
import json

## Train the SVM model in Azure ML
First refactor your training code to a script. See [train.py](train.py).

In [ ]:
# get Azure ML workspace
from azureml.core import Workspace, Experiment, Run, Datastore

ws = Workspace.from_config("/path/to/aml_workspace_config.json")
ds = ws.get_default_datastore()
print(ds.container_name)

model_name = 'your_model_name'
experiment_name = 'your_experiment_name'
compute_name = 'your_training_compute_target'

compute_target = ws.compute_targets[compute_name]

print("datastore:", ds.container_name, ", compute:", type(compute_target))

In [ ]:
from azureml.train.estimator import Estimator

exp = Experiment(workspace=ws, name=experiment_name)

script_folder = './'
script_params = {
    '--data-store': ds.as_mount(),
    '--process-images': 'True',
    '--target-image-size': 360,
    '--model-name': model_name,   
}

est = Estimator(source_directory=script_folder,
                script_params=script_params,
                compute_target=compute_target,
                entry_script='train.py',
                conda_packages=['scikit-learn', 'Pillow'])

run = exp.submit(config=est)

In [ ]:
# get training status
from azureml.widgets import RunDetails

RunDetails(run).show()

In [ ]:
# wait for training to complete, then print metrics
run.wait_for_completion(show_output=False)
print(run.get_metrics())

## Download the trained model, and run some tests
Don't put the downloaded model in the same folder as source because AML will zip the folder when submitting for training

In [ ]:
import os 
import utils

downloadPath = '/path/to/downloaded_models'
os.makedirs(downloadPath, exist_ok=True)

run.download_file(name='outputs/' + model_name, output_file_path=downloadPath)

In [ ]:
import pickle
from sklearn.externals import joblib

testsvm = joblib.load(os.path.join(downloadPath, model_name))

In [ ]:
import numpy as np

img=Image.open('/path/to/test_imagefile')
img_arr = np.array(utils.processImg(img, targetimagesize)).flatten()
pred = testsvm.predict([img_arr])
print(pred)

## If the model is good, register it

In [ ]:
model = run.register_model(
    model_name=model_name, #register from outputs folder in the run
    model_path='outputs/' + model_name, #this must match the file you output in the training script
    tags = {"imagesize":script_params['--target-image-size']}) 
print(model.name, model.id, model.version, sep = ',')

## Create a Docker image for the model
1. Create [score.py](score.py) that implements init() and run(), it calls the model to run predictions
2. Create [env.yml](env.yml) which contains the conda packages this model depends on

In [ ]:
from azureml.core.image import ContainerImage

docker_image_name = 'your_docker_image_name'
# create a Docker image that builds a web service 
image_config = ContainerImage.image_configuration(execution_script='score.py', 
                                                  runtime='python', 
                                                  conda_file='env.yml', 
                                                  dependencies=['utils.py'], 
                                                  tags={'mltype':'svm'})
image = ContainerImage.create(workspace=ws, 
                              name=docker_image_name, 
                              models=[model], 
                              image_config=image_config)
#image = ContainerImage(workspace=ws, name=docker_image_name, version='2')
image.wait_for_creation(show_output=True)

## Deploy to Azure Container Instance as Web Service

In [ ]:
from azureml.core.webservice import Webservice, AciWebservice

webservice_name = 'your_webservice_name'

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=4, 
                                               tags={"method" : "svm"}, 
                                               description='image classification')
#
# if using existing image rather than the one built above
# image = ContainerImage(workspace=ws, name=docker_image_name, version='#your_version')
#
service = Webservice.deploy_from_image(workspace=ws, 
                                       name=webservice_name, 
                                       image=image, 
                                       deployment_config=aciconfig)
service.wait_for_deployment(show_output=True)
print(service.scoring_uri)

#
# scoring_uri will be http://someip/score
# Test the service by issuing a web request
# http://someip/score?url=url_to_test_image
#

## Delete the Web Service

In [ ]:
from azureml.core.webservice import Webservice, AciWebservice

webservice_name = 'your_webservice_name'

service = Webservice(workspace=ws, name=webservice_name)
service.delete()
